Analysis

In [648]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [649]:
import pandas as pd

In [1164]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [1165]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [1166]:
def extract(l,x):
    for i in range(5):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0,Dim=5):
    if Dim==5:
        d=1
    elif Dim==6:
        d=2
    s=-(ν+m)//d
    l=extract(l,ν)
    if m:
        l=extract(l,m)
    #Dirac masses
    for ss in [s,-s]:
        pr=[ p for  p in get_permuations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        l=remove(l,pr)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    return list(np.unique(l))

def count_massless_fermions(l,ν,m=0,Dim=5):
    return len( get_massless_fermions(l,ν,m,Dim=Dim) )

In [1167]:
l=[2, -3, -10, 13, 13, -15]
l=[3, -4, -21, 25, 25, -28]
l=[8, 9, 11, -13, -20, -20, 25]
l=[2, 2, -3, 12, -14, -18, 19]
l=[5, 5, 7, -8, -12, -12, 15]
#l=[6, 8, 11, -14, -14, -18, 21]
#l=[3, 3, -8, -15, 20, 20, -23]
#l=[1, -3, 6, -7, -14, 21, 21, -25]
#l=[1, 1, -4, -5, 9, 9, 9, -10, -10]
l=[3, 3, -6, -9, 10, 12, -15, -15, 17]
l=[1, 3, -7, -9, 13, 24, -27, -27, 29]
#XD
l=[2, -5, -5, -5, 7, 8, 8, -10]

In [1168]:
get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)==[9]

True

In [1169]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2

True

In [1170]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1

True

In [1171]:
count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0

True

In [1172]:
count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3

True

In [1173]:
get_massless_fermions( [2, -4, 7, 9, -18, -18, 22],ν=-18 )==[2,7]

True

In [1174]:
# XDirac with two triplets and Dim 6
get_massless_fermions([2, -7, -8, -8, -8, 18, 18, 18, -25],-8,m=18,Dim=6)==[-25]

True

In [1175]:
def massfer(row,label='DarkDirac',nmax=10,Dim=5):        
    # 
    if label=='DarkDirac' or label=='DarkMajor':
        if label=='DarkDirac':
            nuR='DD'
        else:
            nuR='DM'
        if row.get(label)>0:
            cdd=[]
            for ν in row.get(nuR):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν,Dim=Dim)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac' or label=='XMajor':
        if label=='XDirac':
            nuR='XD'
        else:
            nuR='XM'
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get(nuR):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m,Dim=Dim)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)
        else:
            return nmax    

In [1176]:
cl['DD_massless']=cl.apply(lambda row:   massfer(row,label='DarkDirac'),axis='columns')
cl['DM_massless']=cl.apply(lambda row:   massfer(row,label='DarkMajor'),axis='columns')
cl['XD_massless']=cl.apply(lambda row:   massfer(row,label='XDirac'   ),axis='columns')
cl['XM_massless']=cl.apply(lambda row:   massfer(row,label='XMajor'   ),axis='columns')

In [1327]:
cl6['DD_massless']=cl6.apply(lambda row: massfer(row,label='DarkDirac',Dim=6),axis='columns')
cl6['DM_massless']=cl6.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['XD_massless']=cl6.apply(lambda row: massfer(row,label='XDirac'   ,Dim=6),axis='columns')
cl6['XM_massless']=cl6.apply(lambda row: massfer(row,label='XMajor'   ,Dim=6),axis='columns')

In [1328]:
#TODO: Mv to general functions
def get_label(model,Dim=5):
    if Dim==5:
        dd=''
        dm=1
    elif Dim==6:
        dd='2'
        dm=2    
    if model=='DarkDirac':
        label='D→nu_R+{}fi+{}fj'.format(dd,dd)
    elif model=='DarkMajor':
        label='D→nu_R+{}fi'.format(2*dm)
    elif model=='XDirac':
        label='X→nu_R+{}fi+{}fj'.format(dd,dd)
    else:
        label='X→nu_R+{}fi'.format(2*dm)
    return model,label

In [1329]:
def sltn_intersection(row,model='DarkDirac',Dim=5):
    model,label=get_label(model,Dim=Dim)
    sltn_int=0
    l=list(itertools.chain(*row['sltn'].values() ) )
    #print(l,label)
    for d in row['nu_R']:
        if d.get(label):
            #print(d)
            intrs=len(np.intersect1d(d.get(label), l ))
            if intrs>sltn_int:
                sltn_int=intrs
    return sltn_int

In [1330]:
cl['total']   =cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl['sltn_len']=-cl['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [1331]:
cl6['total']   =cl6['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl6['sltn_len']=-cl6['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [1332]:
row=cl[cl['solution'].astype(str)=='[1, 4, 5, 5, -8, -10, -10, 13]'].reset_index(drop=True).loc[0].to_dict()

In [1333]:
sltn_intersection(row,model='DarkMajor')

2

## Filter unique solutions

Pure solutions

In [1354]:
models.index

RangeIndex(start=0, stop=15, step=1)

In [1458]:
dim=[5,6]
#d=dim[0]
for d in dim:
    if d==5:
        fl=pd.DataFrame()
        clc=cl.copy()
    else:
        fl6=pd.DataFrame()
        clc=cl6.copy()
    models=pd.DataFrame([
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']>0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']> 0) & (clc['XMajor']>0))}
    ])
    #i=0
    for i in models.index:
        mdl=models.loc[i,'model']
        tmp=clc[models.loc[i,'condition']].reset_index(drop=True)
        if not tmp.empty:
            tmp['sltn_len']=-tmp.apply(lambda row: sltn_intersection(row, model=mdl,Dim=d ) ,axis='columns')
            massless={'DarkDirac':'DD_massless','DarkMajor':'DM_massless','XDirac':'XD_massless','XMajor':'XM_massless'}
            if d==5:
                fl=fl.append(
                    tmp.sort_values(['n',massless[mdl],'sltn_len','total']
                     ).drop_duplicates(['n',massless[mdl],'sltn_len']).reset_index(drop=True)
                ).reset_index(drop=True)
                fl['str']=fl['solution'].astype(str)
                fl=fl.drop_duplicates('str').drop('str',axis='columns').reset_index(drop=True)                 
            if d==6:
                fl6=fl6.append(
                    tmp.sort_values(['n',massless[mdl],'sltn_len','total']
                     ).drop_duplicates(['n',massless[mdl],'sltn_len']).reset_index(drop=True)
                ).reset_index(drop=True)
                fl6['str']=fl6['solution'].astype(str)
                fl6=fl6.drop_duplicates('str').drop('str',axis='columns').reset_index(drop=True)                                 

In [1459]:
fl=fl[fl.apply(lambda r:min([r['DD_massless'],r['DM_massless'],r['XM_massless'],r['XD_massless']])<=2,axis='columns' )
     ].sort_values(['n',massless[mdl],'sltn_len','total'])[
       ['n','l','k','gcd','solution','DD', 'DM', 'XD', 'XM', 'DD_massless',
       'DM_massless','XD_massless', 'XM_massless','total','sltn_len','nu_R']].reset_index(drop=True)
fl

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[2, -2]","[-1, 0]",4,"[1, -2, -3, 5, 5, -6]",[5],0,0,0,0,10,10,10,22,-1,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]"
1,7,"[-1, 0]","[-3, -4, -3]",6,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,1,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
2,7,"[-4, -6]","[1, 0, -1]",3,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,1,1,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]"
3,7,"[1, 0]","[-1, 1, -3]",1,"[5, 5, 7, -8, -12, -12, 15]","[-12, 5]",0,0,0,2,10,10,10,64,-2,"[{'D→nu_R+fi+fj': [-12, 5, 7]}]"
4,7,"[-1, 1]","[2, -7, -6]",3,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]"
5,7,"[2, 0]","[-1, -4, -3]",3,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]"
6,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]"
7,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,1,2,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]"
8,8,"[-1, -4, -3]","[2, 3, 4]",2,"[2, 4, 4, 5, -9, -9, -10, 13]","[-9, 4]",0,0,0,2,10,10,10,56,-2,"[{'D→nu_R+fi+fj': [-9, 4, 5]}]"
9,8,"[1, 0, -3]","[1, 2, -2]",2,"[1, 4, 5, 5, -8, -10, -10, 13]",0,"[-10, 5]",0,0,10,1,10,10,56,-2,"[{'D→nu_R+2fi': [-10, 5]}]"


In [1460]:
fl['solution'].apply(lambda l: np.abs(l).max()).max()

31

In [1461]:
fl6=fl6[fl6.apply(lambda r:min([r['DD_massless'],r['DM_massless'],r['XM_massless'],r['XD_massless']])<=2,axis='columns' )
       ].sort_values(['n',massless[mdl],'sltn_len','total'])[
       ['n','l','k','gcd','solution','DD', 'DM', 'XD', 'XM', 'DD_massless',
       'DM_massless','XD_massless', 'XM_massless','total','sltn_len','nu_R']].reset_index(drop=True)

In [1462]:
fl6

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[-1, 1]","[4, -2]",3,"[1, 1, 1, -4, -4, 5]",0,[1],0,0,10,0,10,10,16,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
1,6,"[-4, 2]","[1, 6]",72,"[1, -4, -4, 9, 9, -11]",0,[-4],0,0,10,0,10,10,38,-1,"[{'D→nu_R+4fi': [-4, 1]}]"
2,6,"[2, 1]","[2, 1]",1,"[1, -4, -8, 14, 14, -17]",[14],0,0,0,2,10,10,10,58,-1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]"
3,7,"[-4, -6]","[1, 0, -1]",3,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",0,0,0,2,10,10,10,28,-2,"[{'D→nu_R+2fi+2fj': [-4, 1, 6]}]"
4,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",0,"[-4, 1]",0,0,10,1,10,10,34,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
5,7,"[7, 5]","[-1, 1, 3]",1,"[2, 2, -4, 7, -8, -8, 9]","[-8, 2]","[-8, 2]",0,0,2,2,10,10,40,-2,"[{'D→nu_R+2fi+2fj': [-8, 2, 7]}, {'D→nu_R+4fi': [-8, 2]}]"
6,7,"[3, 1]","[2, 3, -1]",1,"[3, 3, 3, -5, -5, -7, 8]",0,0,[-5],0,10,10,0,10,34,-1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]"
7,7,"[-3, 0]","[-1, 1, 2]",2,"[4, 4, 5, -7, -8, -9, 11]",[4],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+2fi+2fj': [-7, 4, 5]}]"
8,7,"[14, 12]","[-4, -1, 10]",176,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]"
9,8,"[1, 2, -1]","[3, -1, 1]",2,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,0,[-7],10,10,10,2,38,-1,"[{'X→nu_R+4fi': [-7, 1]}]"


In [1464]:
fl6['solution'].apply(lambda l: np.abs(l).max()).max()

32

In [1463]:
#fl['Ref']=''

In [1466]:
#kk=fl[['n','l','k','solution','n','DD','DM','XD','XM']]
kk=fl6[['n','l','k','solution','gcd','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ])   )

In [1430]:
row=cl[cl.sltn.str[3].fillna('').apply(len)==2].reset_index(drop=True).loc[0].to_dict()

In [1429]:
sltn_intersection(row, model='DarkDirac',Dim=5 )

2

In [1431]:
fl

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[2, -2]","[-1, 0]",4,"[1, -2, -3, 5, 5, -6]",[5],0,0,0,0,10,10,10,22,-1,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]"
1,7,"[-1, 0]","[-3, -4, -3]",6,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,1,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
2,7,"[-4, -6]","[1, 0, -1]",3,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,1,1,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]"
3,7,"[1, 0]","[-1, 1, -3]",1,"[5, 5, 7, -8, -12, -12, 15]","[-12, 5]",0,0,0,2,10,10,10,64,-2,"[{'D→nu_R+fi+fj': [-12, 5, 7]}]"
4,7,"[-1, 1]","[2, -7, -6]",3,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]"
5,7,"[2, 0]","[-1, -4, -3]",3,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]"
6,7,"[4, -5]","[3, 4, -4]",1,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]"
7,8,"[1, -6, -9]","[1, 0, -2]",2,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,1,2,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]"
8,8,"[-1, -4, -3]","[2, 3, 4]",2,"[2, 4, 4, 5, -9, -9, -10, 13]","[-9, 4]",0,0,0,2,10,10,10,56,-2,"[{'D→nu_R+fi+fj': [-9, 4, 5]}]"
9,8,"[1, 0, -3]","[1, 2, -2]",2,"[1, 4, 5, 5, -8, -10, -10, 13]",0,"[-10, 5]",0,0,10,1,10,10,56,-2,"[{'D→nu_R+2fi': [-10, 5]}]"


In [1413]:
fl['solution'].apply(lambda l: np.abs(l).max()).max()

23

In [1438]:
cl.shape

(4955, 20)

In [1437]:
cl[cl['solution'].apply(lambda l: np.abs(l).max())<=23].shape

(1823, 20)

In [1415]:
cl6['solution'].apply(lambda l: np.abs(l).max()).max()

32